In [470]:
import pandas as pd 

In [471]:
data = pd.read_csv('C:Downloads/best_salesman_homework.csv')

## EDA

In [472]:
data.head()

,client_account_id,date,event_name,manager_id,manager_nickname
0,0,2022-05-09,first_touch,1.0,Justin Beiber
1,1,2022-03-21,first_touch,3.0,Joe Biden
2,2,2022-04-18,first_touch,2.0,Kylie Jenner
3,3,2022-02-07,first_touch,2.0,Kylie Jenner
4,4,2022-04-08,first_touch,1.0,Justin Beiber


In [473]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3183 entries, 0 to 3182
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   client_account_id  3183 non-null   int64  
 1   date               3183 non-null   object 
 2   event_name         3183 non-null   object 
 3   manager_id         3183 non-null   float64
 4   manager_nickname   3183 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 124.5+ KB


In [474]:
data.describe()

,client_account_id,manager_id
count,3183.000000,3183.000000
mean,1494.974552,2.101162
std,861.116990,0.820737
min,0.000000,1.000000
25%,753.500000,1.000000
50%,1492.000000,2.000000
75%,2245.500000,3.000000
max,2986.000000,3.000000


In [475]:
data['event_name'].value_counts()

first_touch    2987
deal            196
Name: event_name, dtype: int64

In [476]:
data['manager_nickname'].value_counts()

Joe Biden        1249
Kylie Jenner     1007
Justin Beiber     927
Name: manager_nickname, dtype: int64

In [477]:
data['client_account_id'].value_counts()

1175    2
1329    2
2751    2
2393    2
2390    2
       ..
1027    1
1028    1
1029    1
1030    1
2986    1
Name: client_account_id, Length: 2987, dtype: int64

In [478]:
data['manager_id'].value_counts()

3.0    1249
2.0    1007
1.0     927
Name: manager_id, dtype: int64

In [479]:
data['date'].value_counts()

2022-05-15    26
2022-04-17    23
2022-05-29    23
2022-02-09    22
2022-01-22    22
              ..
2021-06-17     1
2021-07-12     1
2021-05-19     1
2021-07-14     1
2021-05-17     1
Name: date, Length: 379, dtype: int64

In [480]:
data.duplicated().sum()

0

## Data processing

In [481]:
data['date']  = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['manager_id'] = data['manager_id'].astype('int')

## First manager

In [482]:
data_mng_1 = data[data['manager_nickname']=='Joe Biden']

In [483]:
data_mng_1 = data_mng_1.set_index('date')

In [484]:
data_mng_1.index.is_monotonic

False

In [485]:
data_mng_1.sort_index(inplace=True)
data_mng_1.index.is_monotonic

True

In [486]:
data_mng_1['manager_id'].value_counts()

3    1249
Name: manager_id, dtype: int64

In [487]:
data_mng_1.head()

,client_account_id,event_name,manager_id,manager_nickname
date,,,,
2021-05-16,2560,first_touch,3,Joe Biden
2021-05-16,1557,first_touch,3,Joe Biden
2021-05-16,1567,first_touch,3,Joe Biden
2021-05-16,659,first_touch,3,Joe Biden
2021-05-17,2982,first_touch,3,Joe Biden


In [488]:
# mng_1_deal = data_mng_1[data_mng_1['event_name']=='deal']
# mng_1_touch = data_mng_1[data_mng_1['event_name']=='first_touch']
mng_1_deal_id = list(data_mng_1[data_mng_1['event_name']=='deal']['client_account_id'])
mng_1_touch_id = list(data_mng_1[data_mng_1['event_name']=='first_touch']['client_account_id'])

In [489]:
len(mng_1_deal_id)+len(mng_1_touch_id)-len(data_mng_1)

0

In [490]:
mng_1_touch_deal_id =[]
for i in mng_1_deal_id:
    if i in mng_1_touch_id:
        mng_1_touch_deal_id.append(i)

In [491]:
import numpy as np
data[np.isin(data, mng_1_touch_deal_id).any(axis=1)]

,client_account_id,date,event_name,manager_id,manager_nickname
186,176,2021-08-31,first_touch,3,Joe Biden
187,176,2021-09-04,deal,3,Joe Biden
274,259,2022-05-15,first_touch,3,Joe Biden
275,259,2022-03-09,deal,3,Joe Biden
350,330,2021-07-11,first_touch,3,Joe Biden
...,...,...,...,...,...
2924,2738,2022-04-18,deal,3,Joe Biden
2937,2751,2021-12-29,first_touch,3,Joe Biden
2938,2751,2022-04-04,deal,3,Joe Biden
3041,2849,2022-04-02,first_touch,3,Joe Biden


In [492]:
len(mng_1_deal)-len(mng_1_touch_deal_id)

57

In [493]:
len(mng_1_touch_deal_id) 

34

In [494]:
len(data_mng_1) 

1249

In [495]:
mng_1_conversion = round((len(mng_1_touch_deal_id)/len(data_mng_1)*100), 3)
print("Conversion of the first manager", str(mng_1_conversion)+'%.')

Conversion of the first manager 2.722%.


In [496]:
def conversion(data):
    data = data.set_index('date')
    data.sort_index(inplace=True)
    mng_1_deal_id = list(data[data['event_name']=='deal']['client_account_id'])
    mng_1_touch_id = list(data[data['event_name']=='first_touch']['client_account_id'])
    if (len(mng_1_deal_id)+len(mng_1_touch_id)-len(data))!=0:
        return print("Error, check the data")
    mng_1_touch_deal_id =[]
    for i in mng_1_deal_id:
        if i in mng_1_touch_id:
            mng_1_touch_deal_id.append(i)
    return round((len(mng_1_touch_deal_id)/len(data)*100), 3)

In [497]:
data_mng_1 = data[data['manager_nickname']=='Joe Biden']
conversion(data_mng_1)

2.722

## Second manager

In [498]:
data_mng_2 = data[data['manager_nickname']=='Kylie Jenner']

In [499]:
conversion(data_mng_2)

1.986

## Third manager

In [500]:
data_mng_3 = data[data['manager_nickname']=='Justin Beiber']
conversion(data_mng_3)

1.187

In [501]:
pd.DataFrame(data=[['Joe Biden', conversion(data_mng_1)],
                  ['Kylie Jenner', conversion(data_mng_2)],
                  ['Justin Beiber', conversion(data_mng_3)]], columns=['manager_nickname', 'conversion, %'],)

,manager_nickname,"conversion, %"
0,Joe Biden,2.722
1,Kylie Jenner,1.986
2,Justin Beiber,1.187


In [502]:
round(conversion(data_mng_1))

3